In [ ]:
import statsmodels.api as sm

def exponential_smoothing(data, alpha, beta, periods):
    n = len(data)
    smoothed_values = [data[0]]
    for i in range(1, n):
        smoothed = alpha * data[i] + (1 - alpha) * (smoothed_values[i - 1] + beta * (data[i] - smoothed_values[i - 1]))
        smoothed_values.append(smoothed)
    forecasted_values = []
    for i in range(1, periods + 1):
        forecasted = alpha * data[-1] + (1 - alpha) * (smoothed_values[-1] + beta * (data[-1] - smoothed_values[-1]))
        forecasted_values.append(forecasted)
        smoothed_values.append(forecasted)
    return forecasted_values
for crypto, df in df_dict.items():
    data = df['C'].values
    returns = (df['C'] - df['C'].shift(1)) / df['C'].shift(1)
    returns = returns.dropna()

    Y = returns
    X = df['C'].shift(1).dropna()
    
    Y = Y.loc[X.index]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    alpha = model.params[0]
    beta = model.params[1]

    hw_forecast_test = []
    for t in range(config['MODEL']['WINDOW_SIZE'], len(data) - config['MODEL']['PREDICT_STEP']):
        train = data[t-config['MODEL']['WINDOW_SIZE']:t]
        pred = exponential_smoothing(train, alpha, beta, config['MODEL']['PREDICT_STEP'])[-1]
        hw_forecast_test.append(pred)
    plt.figure(figsize=(12, 6))
    plt.plot(data[config['MODEL']['WINDOW_SIZE'] + config['MODEL']['PREDICT_STEP']:], label='True Prices', color='blue')
    plt.plot(hw_forecast_test, label='Forecasted Prices', color='red')
    plt.title(crypto + ': HW Exponential Smoothing Test Set Forecast')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()
    print(crypto, eval(data[config['MODEL']['WINDOW_SIZE'] + config['MODEL']['PREDICT_STEP']:], hw_forecast_test))
